In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.covariance import MinCovDet
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
#df2=pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
target_df=pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test_df =pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_df.head(1)

In [ ]:
"""
from scipy import stats
nums=[]   
for i in range(206):
    a=target_df2.iloc[:,i]
    num=0
    col=[]
    for j in train_df2:
        
        one_data=train_df2.loc[a[a==1].index,j]
        zero_data=train_df2.loc[a[a==0].index,j]
        a2=stats.mannwhitneyu(one_data,zero_data, alternative='two-sided')
        if a2.pvalue <0.05:
            num+=1
            col.append(j)
    if num ==0:
        nums.append(i)
    print(i,"は有意差が生じる説明変数が",num,"個で",col)
    print("---next_target------")
print("有意差が生じない目的変数は",nums)
"""


In [ ]:
tar=target_df.set_index('sig_id')
tar2=tar.sum()
tar2.plot(kind="hist")
#tar2.plot(kind="box")

ラベルの数が結構まちまちなので、３つぐらいにグループ分けしてモデルを作ってみる

In [ ]:
#ダウンサンプリング #df2はtargetでなければいけいない
def down_sampling(df1,df2,percent=0.5,kotei=True):
    d1 = df1.copy()
    d2 = df2.copy()
    num = int(len(df1)*percent)
    if ('sig_id' in d1.columns) or ('sig_id' in d2.columns):
        d1 = d1.set_index('sig_id')
        d2 = d2.set_index('sig_id')
    
    dd1 = d1.sample(n=num)
    dd2 = d2.loc[dd1.index]
    return dd1,dd2

In [ ]:
def proceccing2(train3,mcd=None,jogen=None):
    train4 = train3.copy()
    if mcd==None:
        mcd =MinCovDet(support_fraction=0.7)
        mcd.fit(train4)
    dis = mcd.mahalanobis(train4)
    
    
    se=pd.Series(dis,index=train4.index)
    
    if jogen==None:
        num =int(len(se)*0.01)
        #    print(num)
        iqr = se.quantile(0.75)-se.quantile(0.25)
        jogen2= se.quantile(0.75) + 1.5*iqr
        se_idx=se[se>jogen2].index
        
    else:
        se_idx=se[se>jogen].index
        jogen2=0
    train4["out_flag"]= 0
    train4.loc[se_idx,"out_flag"]=1
    return train4,mcd,jogen2

In [ ]:
#ダミー変数化を行い。主成分分析でcomp分に列を纏める
def proceccing(d,comp=10,test=False,mcd=None,):
    df = d.copy()
    data={"trt_cp":0,'ctl_vehicle':1}
    df['cp_type']=df['cp_type'].map(data)
    data={"D1":0,'D2':1}
    df['cp_dose']=df['cp_dose'].map(data)
    #df=df.drop('sig_id',axis=1)
    pca = PCA(n_components=comp,random_state=0,whiten=False)
    train2 = df.set_index('sig_id')
    pca.fit(train2)
    train3=pd.DataFrame(pca.transform(train2))
    train3.index= train2.index
    
    return train3

In [ ]:
#引数の仕様df1とdf2はインデックスでつなげる事が出来る。
def under_sampling(df1,df2):#df2がtargetの1列のシリーズ
    d1 = df1.copy()
    d2 = df2.copy()
    
    #positiveの行を全て抜き出す。
    dd2_1 = d2[d2==1]
    num_1 = len(dd2_1)#1の行の総数
    
    a = d2[d2==0]
    num_0 =len(a)#0の行の数
    if num_1*20 < num_0:
        num =num_1*20
    else:
        num = num_0
    
    ##negativeの行を全て抜き出す。
    dd2_0 = d2[d2==0]
    if num != 0:
        #0の行をランダムに抜き出す（件数はnumの10倍)
        dd2_0_2 = dd2_0.sample(random_state=10,n=num)
    else:
        #1
        dd2_0_2 = dd2_0.sample(random_state=10,n=100)
    
    #d1からpositive分とネガティブ分のデータを抜き出す
    d1_1 =d1.loc[dd2_1.index]
    d1_0 = d1.loc[dd2_0_2.index]
    
    
    d11 = pd.concat([d1_1,d1_0])#訓練データの結合（行方向）
    d22 = pd.concat([dd2_1,dd2_0_2])#訓練データの結合（行方向）
    return d11,d22

In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.optimizers import SGD,Adam
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import StandardScaler

train_df2 = proceccing(train_df,comp=100)#pca
train_df2,mcd,jogen=proceccing2(train_df2)#外れ値フラグを立てる
target_df2 = target_df.set_index('sig_id')
    
sc_x =StandardScaler()
#sc_t =StandardScaler()
sc_df2=sc_x.fit_transform(train_df2)
#sc_target2=sc_t.fit_transform(target_df2)



In [ ]:
#t=target_df2.iloc[:,i]
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=101))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))
model.add(Dense(units=32, activation='relu', input_dim=64))
model.add(BatchNormalization())
    #model.add(Dropout(rate=0.5))
model.add(Dense(units=target_df2.shape[1], activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01))
    #model.fit(train_x, train_t, epochs=50, batch_size=10) # エポック50, ミニバッチ 10
ep=30
batch_size=128
stack =model.fit(sc_df2,target_df2, epochs=ep, batch_size=batch_size,validation_split=0.2,shuffle=True)

In [ ]:
"""scores=[]
for i in range(206):
    t=target_df2.iloc[:,i]
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_dim=100))
    model.add(BatchNormalization())
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=32, activation='relu', input_dim=64))
    model.add(BatchNormalization())
    #model.add(Dropout(rate=0.5))
    model.add(Dense(units=1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01))
    #model.fit(train_x, train_t, epochs=50, batch_size=10) # エポック50, ミニバッチ 10
    ep=15
    batch_size=256
    stack =model.fit(sc_df2,t, epochs=ep, batch_size=batch_size,validation_split=0.2,shuffle=True,verbose=0)
    scores.append(stack.history['val_loss'][-1])
    print(stack.history['loss'][-1],stack.history['val_loss'][-1])
    print("next")
    # エポック50, ミニバッチ 10
print("finish")
"""

In [ ]:
import matplotlib.pyplot as plt
x = list(range(ep))
plt.plot(x[2:], stack.history['loss'][2:], label="loss")
plt.title("loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.plot(x[2:], stack.history['val_loss'][2:], label="val_loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
submit = test_df[['sig_id']]
submit = submit.copy()
li=submit.values.reshape(-1)
test_df2=proceccing(test_df,comp=100)
test_df2,m,j=proceccing2(test_df2,mcd,jogen)

test_df2 = sc_x.fit_transform(test_df2)
pred=model.predict(test_df2)
submit2=pd.DataFrame(pred,index=li)
submit2=submit2.reset_index()
submit2.columns=target_df.columns
submit2.head()

#　Xgboostによる解

In [ ]:
"""
train_df2 = proceccing(train_df,comp=100)
target_df2 = target_df.set_index('sig_id')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
scores=[]
models=[]
avgs=[]
target_num = target_df2.shape[1]
log_loss=[]
col=["name","accuracy","train_sum","train_num","precision","recall","f1_score"]
for i in range(target_num):
    
    #while True:
    #train,target=down_sampling(train_df2,target_df2,percent=0.3)


    y = target_df2.iloc[:,i].values
    print(y.shape)
    x = train_df2.values
    
    
    dtrain = xgb.DMatrix(x, label=y)
    #dtest = xgb.DMatrix(X_test, label=y_test)
    # 学習用のパラメータ
    xgb_params = {
        'max_depth': 5,
        # 二値分類問題
        'objective': 'binary:logistic',
        # 評価指標
        'eval_metric': 'logloss',
        
    }
    #evallist = [(dvalid, 'eval'), (dtrain, 'train')]
    # モデルを学習する
    bst = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=10  # 学習ラウンド数は適当
                    
                    )
    
    #log_lossの計算
    #１の確率
    proba = bst.predict(dtrain)
    t=0
    for j in range(len(y)):
        tar = y[j]
        if tar ==0:
            p = np.log(1-proba[i])
        else:
            p = np.log(proba[i])
        t += p
    t = -t/len(y)
    log_loss.append(t)
            
        
    
    
    
    #score=[target_df2.columns[i],accuracy_score(y_pred=model.predict(X_test),y_true=y_test),y_train.sum(),len(y_train),score_p,score_r,score_f]
    #scores.append(score)
    print(i,"end_logloss",t)
#print("total_avg",np.mean(avgs))
"""

In [ ]:
"""

import matplotlib.pyplot as plt
for i in result:
    if i =="name":
        continue
    result[i].plot(kind="hist",title=i)
    plt.show()
"""

In [ ]:
"""
submit = test_df[['sig_id']]
submit = submit.copy()
test_df2=proceccing(test_df)
#test_df2.head()
for i in range(target_df2.shape[1]):
    try:
        y1 = models[i].predict_proba(test_df2)
        y1 = y1[:,1]
        #xgboost
        #dtest = xgb.DMatrix(test_df2)
        #y1 = models[i].predict(dtest)

        #y1 = models[i].predict(test_df2)
       
        #y11=list(map(lambda x:round(x[1],5),y1))
        y111 =list(map(lambda x: 0.0001 if x==0 else x,y1))
        submit[target_df2.columns[i]]=y111
    except:
        print(i)
submit.shape
"""

In [ ]:
submit2.to_csv('submission.csv',index=False)